In [1]:
import tensorflow as tf
import numpy as np
from osgeo import gdal
import os
import random
import pickle
import subprocess

In [2]:
os.chdir('/scripts/data')
subprocess.Popen(['tensorboard', '--logdir=../tb_logs'])
learning_rate = 0.001
num_steps = 500
batch_size = 16
#display_step = 10
num_epochs = 100

# Network Parameters
edgelength = 64
#num_pixels = edgelength * edgelength
num_chans = 4
#num_input = num_pixels * num_chans # MNIST data input (img shape: 28*28)
num_classes = 4 # MNIST total classes (0-9 digits)
dropout = 0.65 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, edgelength, edgelength, num_chans])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [3]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    # x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    
    # Convolution Layer
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    
    # Convolution Layer
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    conv5 = maxpool2d(conv5, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv5, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [4]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, num_chans, 32])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc4': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wc5': tf.Variable(tf.random_normal([3, 3, 256, 512])),
    
    'wd1': tf.Variable(tf.random_normal([2*2*512, 1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bc4': tf.Variable(tf.random_normal([256])),
    'bc5': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter('/scripts/tb_logs/train/1')
test_writer = tf.summary.FileWriter('/scripts/tb_logs/test/1')

In [5]:
init = tf.global_variables_initializer()

In [6]:
def train_test_split(xloc, yloc, p):
    X_file = pickle.load( open(xloc, "rb" ))
    y_file = pickle.load( open(yloc, "rb" ))
    
    y_file = np.array(y_file)-1
    n_values = np.max(y_file)+1
    y_file = np.eye(n_values)[y_file]
    
    c = list(zip(X_file, y_file))
    random.shuffle(c)
    
    X_file, y_file = zip(*c)
    
    number = int(len(y_file) * p)
    
    X_file_tr = X_file[:number]
    y_file_tr = y_file[:number]
    
    X_file_te = X_file[number:]
    y_file_te = y_file[number:]    
    
    return X_file_tr, y_file_tr, X_file_te, y_file_te

def generate_batches(Xs, ys, bs):
    #X_file = pickle.load(open("images.pkl", "rb" ))
    #y_file = pickle.load(open("classes.pkl", "rb" ))
    
    X_batches = []
    y_batches = []    
    Xss = []
    yss = []
    
    for i, x in enumerate(Xs):
        S = gdal.Open(x)
        A = S.ReadAsArray()
        
        where_are_NaNs = np.isnan(A)
        A[where_are_NaNs] = -40.
        
        #print(np.min(A))
            
        A = np.transpose(A)
        y = ys[i]
        
        Xss.append(A)
        yss.append(y)
        
        if (i % bs == 0) and (i != 0):
            X_batches.append(Xss)
            y_batches.append(yss)
            
            Xss = []
            yss = []
            
    X_batches[0] = X_batches[0][:-1]
    y_batches[0] = y_batches[0][:-1]
    return X_batches, y_batches

def generate_test(Xs, ys):
    Xss = []
    yss = []
    for i, X in enumerate(Xs):
        S = gdal.Open(X)
        A = S.ReadAsArray()
        where_are_NaNs = np.isnan(A)
        A[where_are_NaNs] = -40.
        
        #print(np.min(A))
        A = np.transpose(A)
        Xss.append(A)
        yss.append(ys[i])
    return Xss, yss

In [7]:
os.chdir('./sumatra_tiles')
train_test_ratio = 0.85
xtr, ytr, xte, yte = train_test_split('/scripts/data/pythondata/images.pkl', '/scripts/data/pythondata/classes.pkl', train_test_ratio)
X_batches, y_batches = generate_batches(xtr, ytr, batch_size)
X_b, y_b = generate_batches(xtr, ytr, batch_size)
xte, yte = generate_test(xte, yte)

In [8]:
# Start training
# step = 0
display_step = 1
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    for e in range(1, num_epochs + 1):
        for i, xb in enumerate(X_b):
            yb = y_b[i]
            #print(np.min(np.array(xb)))
            sess.run(train_op, feed_dict={X: xb, Y: yb, keep_prob: dropout})            
            
        if e % display_step == 0 or e == 1:
            summary, loss, acc = sess.run([merged, loss_op, accuracy], feed_dict={X: xb,
                                                                 Y: yb,
                                                                 keep_prob: 1.0})
            train_writer.add_summary(summary, e)

            print("Step " + str(e) + ", Minibatch Loss= " + \
                    "{:.4f}".format(loss) + ", Training Accuracy= " + \
                    "{:.3f}".format(acc))

            # Calculate accuracy for 256 MNIST test images
            print("Testing Accuracy:")
            summary, acc = sess.run([merged, accuracy], feed_dict={X: xte,
                                      Y: yte,
                                      keep_prob: 1.0})
            test_writer.add_summary(summary, e)
            print(acc)
            

Step 1, Minibatch Loss= 1155619456.0000, Training Accuracy= 0.750
Testing Accuracy:
0.616564
Step 2, Minibatch Loss= 359117056.0000, Training Accuracy= 0.688
Testing Accuracy:
0.754601
Step 3, Minibatch Loss= 109623712.0000, Training Accuracy= 0.750
Testing Accuracy:
0.730061
Step 4, Minibatch Loss= 68466520.0000, Training Accuracy= 0.750
Testing Accuracy:
0.723926
Step 5, Minibatch Loss= 31327096.0000, Training Accuracy= 0.812
Testing Accuracy:
0.739264
Step 6, Minibatch Loss= 6303688.0000, Training Accuracy= 0.938
Testing Accuracy:
0.757669
Step 7, Minibatch Loss= 5654132.0000, Training Accuracy= 0.938
Testing Accuracy:
0.748466
Step 8, Minibatch Loss= 13038956.0000, Training Accuracy= 0.875
Testing Accuracy:
0.766871
Step 9, Minibatch Loss= 61984.0000, Training Accuracy= 0.938
Testing Accuracy:
0.769939
Step 10, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Testing Accuracy:
0.809816
Step 11, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Testing Accuracy:
0.788344
Step 12, Min

0.846626
Step 97, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Testing Accuracy:
0.871166
Step 98, Minibatch Loss= 1780264.0000, Training Accuracy= 0.938
Testing Accuracy:
0.800614
Step 99, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Testing Accuracy:
0.815951
Step 100, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Testing Accuracy:
0.849693
